In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding,GlobalAveragePooling1D,Dense,Bidirectional,LSTM
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import to_categorical
import numpy as np
import json

In [2]:
# obtain initial data
datastore = []
with open(r"News-Headlines-Dataset-For-Sarcasm-Detection/Sarcasm_Headlines_Dataset.json", "r") as f:
    for line in f:
        datastore.append(json.loads(line))


In [3]:
# generate data and labels
# generate lists of raw data we need (can also filter out all punctuation as part of preprocessing as well)
labels = []
sentences = []
urls = []

for item in datastore:
    labels.append(item['is_sarcastic'])
    sentences.append(item['headline'])
    urls.append(item["article_link"])

sentences = sentences[:3000]

In [4]:
# for generating text, we only need the raw text data

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
tr_seq = tokenizer.texts_to_sequences(sentences)
unique_w = len(tokenizer.word_index) + 1 # len(tokenizer.word_index) 
# refer to the number of unique words, but since we also want to include 0 as the tokenization of a imaginary word, 
# we add 1 (we padded with 0, however in order to do prediction, we need to make sense of every token in the input sequence, thus we must put "0" as additional tokenizaed word in the dictionary)

In [5]:
raw_data = []
for line in tr_seq:
    for i in range(1, len(line)):
        raw_data.append(line[:i+1])
# now pad these raw data, based on the largest sequence len
max_seq_len = max([len(x) for x in raw_data])
pad_raw_data = pad_sequences(raw_data, padding="pre", maxlen=max_seq_len)

In [6]:
tr_set = pad_raw_data[:,:-1]
tr_label = to_categorical(pad_raw_data[:,-1], num_classes=unique_w)

In [7]:
tr_try = tr_set[:5000]
label_try = tr_label[:5000]

In [8]:
tr_set.shape

(27776, 28)

In [9]:
model = Sequential()
model.add(Embedding(unique_w, input_length=max_seq_len-1, output_dim=240)) # max_seq_len - 1 because we drop 1 entry for label
model.add(Bidirectional(LSTM(150)))
model.add(Dense(unique_w, activation="softmax"))

adam = Adam(lr=0.01)
model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])

c:\Python39\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 28, 240)           2172000   
                                                                 
 bidirectional (Bidirectiona  (None, 300)              469200    
 l)                                                              
                                                                 
 dense (Dense)               (None, 9050)              2724050   
                                                                 
Total params: 5,365,250
Trainable params: 5,365,250
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(tr_set, tr_label, epochs=50, verbose=1)

Epoch 1/50
868/868 [==============================] - 74s 81ms/step - loss: 8.7506 - accuracy: 0.0278
Epoch 2/50
868/868 [==============================] - 67s 78ms/step - loss: 10.0132 - accuracy: 0.0238
Epoch 3/50
868/868 [==============================] - 64s 74ms/step - loss: 9.3725 - accuracy: 0.0304
Epoch 4/50
868/868 [==============================] - 54s 62ms/step - loss: 8.7392 - accuracy: 0.0387
Epoch 5/50
868/868 [==============================] - 56s 65ms/step - loss: 7.4324 - accuracy: 0.0499
Epoch 6/50
868/868 [==============================] - 55s 63ms/step - loss: 6.7807 - accuracy: 0.0636
Epoch 7/50
868/868 [==============================] - 52s 59ms/step - loss: 6.2657 - accuracy: 0.0846
Epoch 8/50
868/868 [==============================] - 50s 57ms/step - loss: 5.6952 - accuracy: 0.1123
Epoch 9/50
868/868 [==============================] - 59s 68ms/step - loss: 5.2660 - accuracy: 0.1468
Epoch 10/50
868/868 [==============================] - 50s 58ms/step - loss: 4.84

# Problem: does not consider adding Punctuation to the auto-generated sentence

In [20]:
seed_text = "it is"

words_to_generate = 12
for _ in range(words_to_generate):
    line_seq = tokenizer.texts_to_sequences([seed_text])
    padded = pad_sequences(line_seq, maxlen=max_seq_len-1, padding="pre")
    predict_probs = model.predict(padded,verbose=0)
    predict_label = np.argmax(predict_probs,axis=1)
    output_word = ""
    for word,index in tokenizer.word_index.items(): # the index (tokenized 
        if predict_label == index:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)


it is it anyways erupt over honker weapons attack on healthcare bill about being


# Prediction For Seen Training Data

In [13]:
test_sent = pad_raw_data[:3,:]
test_tr = test_sent[:,:-1]
test_label = test_sent[:,-1]

In [14]:
test_sent

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 3484,  235],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 3484,  235, 1485],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 3484,  235, 1485, 3485]])

In [15]:
test_tr


array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 3484],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 3484,  235],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 3484,  235, 1485]])

In [16]:
test_label

array([ 235, 1485, 3485])

In [17]:

pred = np.argmax(model.predict(test_tr,verbose=0),axis=1)
pred

array([ 235, 1485, 3485], dtype=int64)

In [18]:
history.history["accuracy"]

[0.027793778106570244,
 0.02376152016222477,
 0.030421946197748184,
 0.038702476769685745,
 0.049935195595026016,
 0.06358006596565247,
 0.08456940948963165,
 0.11225518584251404,
 0.14678139984607697,
 0.18379175662994385,
 0.21313363313674927,
 0.23898328840732574,
 0.2705933153629303,
 0.28974655270576477,
 0.2983871102333069,
 0.32391273975372314,
 0.3454781174659729,
 0.3625071942806244,
 0.37384793162345886,
 0.38328051567077637,
 0.3908050060272217,
 0.40081363916397095,
 0.4190308153629303,
 0.4259432554244995,
 0.42709532380104065,
 0.43523186445236206,
 0.45337700843811035,
 0.46140551567077637,
 0.4696860611438751,
 0.476670503616333,
 0.48635512590408325,
 0.4921875,
 0.5011880993843079,
 0.5054003596305847,
 0.5207013487815857,
 0.5242295265197754,
 0.5058683753013611,
 0.5247696042060852,
 0.5343822240829468,
 0.5358942747116089,
 0.5477750301361084,
 0.5490711331367493,
 0.5297738909721375,
 0.31822437047958374,
 0.4216949939727783,
 0.48268288373947144,
 0.5134648680686